# SQLAlchemy ORM Ref
[*v0.1*]

Notebook for prototyping SQLAlchemy 2.0's new ORM syntax.

# Setup

Cells in this section handle notebook setup, like importing packages and functions/vars from scripts.

## Imports

Import `stdlib` packages (i.e. `pathlib.Path`) and package dependencies.

### stdlib

In [672]:
from pathlib import Path
import json
from typing import Any, Optional, Union, Type
from typing_extensions import Annotated
import datetime
import decimal
import uuid
import enum

### Dependencies

Packages installed with `pip` (or some equivalent tool)

In [673]:
import sqlalchemy as sa
from sqlalchemy import Table, Column, create_engine, MetaData, func
from sqlalchemy.orm import (
    DeclarativeBase,
    Mapped,
    mapped_column,
    relationship,
    Session,
    registry,
)

## Use with print to demo table SQL, i.e. print(CreateTable(SomeClass.__table__))
from sqlalchemy.schema import CreateTable

## Global Vars

Variables for use throughout the notebook

In [674]:
nb_log: bool = True
nb_verbose: bool = False

## Functions

Notebook-level functions. These differ from functions imported from scripts in that they are either prototypes, or functions meant only for the notebook.

### Notebook Functions

### Prototype

## Classes

Notebook-level classes. These differ from classes/models imported from scripts in that they are either prototypes, or functions meant only for the notebook.

## SQLAlchemy Setup

### SQLAlchemy Functions

In [675]:
def print_sql(
    _class: Union["User", "UserAnnotated", "Address", "AddressAnnotated"] = None
) -> None:
    """Print the SQL generated from a Mapped SQLAlchemy class.

    Put classnames in the Union[] list, and use quotes. This enabled type hinting before
    the class is declared.
    """
    if not _class:
        raise ValueError("Missing class input.")

    print(
        f"Generated SQL for class [{_class.__tablename__}]:\n{CreateTable(_class.__table__)}"
    )

### SQLAlchemy Classes

- [Docs: DeclarativeBase](https://docs.sqlalchemy.org/en/20/orm/declarative_styles.html#orm-declarative-generated-base-class)
- [Docs: MetaData object](https://docs.sqlalchemy.org/en/20/tutorial/metadata.html#tutorial-working-with-metadata)
- [Docs: Working with registry](https://docs.sqlalchemy.org/en/20/orm/declarative_styles.html#orm-declarative-generated-base-class)

In [676]:
## Registry object stores mappings & config hooks
reg = registry()

In [677]:
## If using a DeclarativeBase class, initialize with Base class's metadata.create_all()
#  Choose one or the other, declarative classes with DeclarativeBase, or a metadata obj
#  and implicit tables


## SQLAlchemy DeclarativeBase is the parent class object table classes will inherit from
#  https://docs.sqlalchemy.org/en/20/orm/declarative_styles.html#using-a-declarative-base-class
class Base(DeclarativeBase):
    registry = reg


## SQLAlchemy MetaData object
#  https://docs.sqlalchemy.org/en/20/tutorial/metadata.html#tutorial-working-with-metadata
# metadata: MetaData = MetaData()

In [678]:
## Type map for class datatypes
#  https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#orm-declarative-mapped-column-type-map

# default type mapping, deriving the type for mapped_column()
# from a Mapped[] annotation
# type_map: dict[Type[Any], sa.types.TypeEngine[Any]] = {
#     bool: sa.types.Boolean(),
#     bytes: sa.types.LargeBinary(),
#     datetime.date: sa.types.Date(),
#     datetime.datetime: sa.types.DateTime(),
#     datetime.time: sa.types.Time(),
#     datetime.timedelta: sa.types.Interval(),
#     decimal.Decimal: sa.types.Numeric(),
#     float: sa.types.Float(),
#     int: sa.types.Integer(),
#     str: sa.types.String(),
#     uuid.UUID: sa.types.Uuid(),
# }

In [679]:
## Compose configurations for mapped columns, i.e. generate an ID or a timestamp
#  https://docs.sqlalchemy.org/en/20/orm/declarative_tables.html#mapping-whole-column-declarations-to-python-types

## Annotate a primary key of type integer
intpk = Annotated[int, mapped_column(primary_key=True)]

## Annotate a timestamp, i.e. for created_at values
timestamp = Annotated[
    datetime.datetime,
    mapped_column(nullable=False, server_default=func.current_timestamp()),
]

## Set a non-nullable annotated field
required_name = Annotated[str, mapped_column(sa.String(30), nullable=False)]
optional_name = Annotated[str, mapped_column(sa.String(30), nullable=True)]


## Example of a mapped class with an Enum class
class Status(enum.Enum):
    PENDING = "pending"
    RECEIVED = "received"
    COMPLETED = "completed"

#### Declarative Mapping

[Docs: Declarative Mapping](https://docs.sqlalchemy.org/en/20/orm/mapping_styles.html)

In [680]:
"""
Create mapped classes. (write more eventually)
"""


class User(Base):
    __tablename__ = "user"

    # metadata: MetaData = metadata
    id: Mapped[int] = mapped_column(sa.Integer, primary_key=True)
    name: Mapped[str] = mapped_column(sa.String(50), index=True, nullable=False)
    fullname: Mapped[Optional[str]] = mapped_column(sa.String)
    ## Create timestamp on the fly
    nickname: Mapped[Optional[str]]
    created_at: Mapped[datetime.datetime] = mapped_column(insert_default=func.now())

    ## Use cascade options to delete relationships when User is deleted
    #  https://docs.sqlalchemy.org/en/20/orm/cascades.html#cascades
    addresses: Mapped[list["Address"]] = relationship(
        back_populates="user", cascade="all, delete-orphan, delete, save-update"
    )


# class UserAnnotated(Base):
#     """
#     Inherit mapped typings/configuration defaults from Annotated types above.
#     """

#     __tablename__ = "user_annotated"

#     id: Mapped[intpk]
#     name: Mapped[required_name]
#     fullname: Mapped[optional_name]
#     ## Timestamp will be generated as a datetime.datetime object
#     created_at: Mapped[Optional[timestamp]]

#     addresses: Mapped[list["Address"]] = relationship(back_populates="user_annotated")


class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(sa.Integer, primary_key=True)
    user_id = mapped_column(sa.ForeignKey("user.id"))
    email_address: Mapped[str] = mapped_column(sa.String, index=True)

    user: Mapped["User"] = relationship(
        back_populates="addresses", cascade="all, delete"
    )


# class AddressAnnotated(Base):
#     __tablename__ = "address_annotated"

#     id: Mapped[intpk]
#     ## Create ForeignKey using config defined in intpk, set as ForeignKey for column
#     user_id: Mapped[intpk] = mapped_column(sa.ForeignKey("user_annotated.id"))
#     ## Override server default, set to UTC timestamp
#     created_at: Mapped[timestamp] = mapped_column(server_default=func.UTC_TIMESTAMP())

In [681]:
class Product(Base):
    __tablename__ = "product"

    id: Mapped[intpk]
    status: Mapped[Status]

In [682]:
print_sql(User)
print_sql(Address)
# print_sql(UserAnnotated)
# print_sql(AddressAnnotated)
print_sql(Product)

Generated SQL for class [user]:

CREATE TABLE "user" (
	id INTEGER NOT NULL, 
	name VARCHAR(50) NOT NULL, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	created_at DATETIME NOT NULL, 
	PRIMARY KEY (id)
)


Generated SQL for class [address]:

CREATE TABLE address (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	email_address VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES "user" (id)
)


Generated SQL for class [product]:

CREATE TABLE product (
	id INTEGER NOT NULL, 
	status VARCHAR(9) NOT NULL, 
	PRIMARY KEY (id)
)




### Engine

In [683]:
## Create connection string for engine
sqlite_db: str = "demo.sqlite"
SQLALCHEMY_DB_URI: str = f"sqlite+pysqlite:///{sqlite_db}"

## Create engine
#  https://docs.sqlalchemy.org/en/20/tutorial/engine.html
engine = create_engine(SQLALCHEMY_DB_URI, echo=True)

In [684]:
## https://docs.sqlalchemy.org/en/20/orm/session_basics.html#basics-of-using-a-session
SessionLocal: Session = Session(bind=engine)

# Operations

Functions & data operations.

In [685]:
## Initialize database, create all metadata
Base.metadata.create_all(bind=engine)

2023-05-27 16:06:32,004 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-27 16:06:32,006 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2023-05-27 16:06:32,007 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-27 16:06:32,015 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2023-05-27 16:06:32,017 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-27 16:06:32,020 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2023-05-27 16:06:32,022 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-27 16:06:32,024 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2023-05-27 16:06:32,025 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-27 16:06:32,027 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("product")
2023-05-27 16:06:32,028 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-27 16:06:32,030 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("product")
2023-05-27 16:06:32,031 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-

## Examples

## Create users, add to session, and commit

Use the following one-liner to check for object existence before adding:

```
if session.query(model).filter(some_filter).count():
    ...
```

For example, to query by a User's name, where `name` = `sandy`:

```
if not session.query(User).filter(User.name == "sandy").count():
    print("User does not exist")

```

In [686]:
## https://docs.sqlalchemy.org/en/20/orm/quickstart.html#create-objects-and-persist

with SessionLocal as sess:
    ## Create users on the fly
    spongebob = User(
        name="spongebob",
        fullname="Spongebob Squarepants",
        addresses=[Address(email_address="spongebob@sqlalchemy.org")],
    )

    sandy = User(
        name="sandy",
        fullname="Sandy Cheeks",
        addresses=[
            Address(email_address="sandy@sqlalchemy.org"),
            Address(email_address="sandy@squirrelpower.org"),
        ],
    )

    patrick = User(name="patrick", fullname="Patrick Start")

    jack = User(
        name="jack",
        fullname="Jack Sparrow",
        addresses=[Address(email_address="captjack@blackpearl.org")],
    )

    ## Create list of users to loop over
    _users: list[User] = [spongebob, sandy, patrick, jack]
    ## Create empty list to store users to commit to db
    add_users: list[User] = []

    ## Loop over _users list
    for _user in _users:
        ## Check if _user already exists
        if not sess.query(User).filter(User.name == _user.name).count():
            ## _user does not exist, add to add_users list
            add_users.append(_user)
        else:
            display(f"Skipping User '{_user.name}', User already exists.")

    ## Add all users in add_users list to session
    sess.add_all(add_users)

    ## Commit to session
    sess.commit()

2023-05-27 16:06:32,167 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-27 16:06:32,174 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname, user.created_at AS user_created_at 
FROM user 
WHERE user.name = ?) AS anon_1
2023-05-27 16:06:32,176 INFO sqlalchemy.engine.Engine [generated in 0.00221s] ('spongebob',)
2023-05-27 16:06:32,179 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname, user.created_at AS user_created_at 
FROM user 
WHERE user.name = ?) AS anon_1
2023-05-27 16:06:32,181 INFO sqlalchemy.engine.Engine [cached since 0.006957s ago] ('sandy',)
2023-05-27 16:06:32,184 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nick

In [687]:
## Select multiple users
#  https://docs.sqlalchemy.org/en/20/orm/quickstart.html#simple-select
stmt_sel_multi = sa.select(User).where(User.name.in_(["sandy", "jack", "patrick"]))

with SessionLocal as sess:
    for user in sess.scalars(stmt_sel_multi):
        display(f"User: {user.fullname}")
        if user.addresses:
            for addr in user.addresses:
                display(f"User address: {addr.email_address}")
        else:
            display(f"User {user.fullname} does not have any addresses.")

2023-05-27 16:06:32,277 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-27 16:06:32,282 INFO sqlalchemy.engine.Engine SELECT user.id, user.name, user.fullname, user.nickname, user.created_at 
FROM user 
WHERE user.name IN (?, ?, ?)
2023-05-27 16:06:32,283 INFO sqlalchemy.engine.Engine [generated in 0.00156s] ('sandy', 'jack', 'patrick')


'User: Jack Sparrow'

2023-05-27 16:06:32,293 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2023-05-27 16:06:32,295 INFO sqlalchemy.engine.Engine [generated in 0.00226s] (4,)


'User address: captjack@blackpearl.org'

'User: Patrick Start'

...

'User: Sandy Cheeks'

2023-05-27 16:06:32,314 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2023-05-27 16:06:32,315 INFO sqlalchemy.engine.Engine [cached since 0.02255s ago] (2,)


'User address: sandy@sqlalchemy.org'

'User address: sandy@squirrelpower.org'

2023-05-27 16:06:32,322 INFO sqlalchemy.engine.Engine ROLLBACK


In [688]:
## Select with JOIN
#  https://docs.sqlalchemy.org/en/20/orm/quickstart.html#select-with-join

sel_join_stmt = (
    sa.Select(Address)
    .join(Address.user)
    .where(User.name == "sandy")
    .where(Address.email_address == "sandy@sqlalchemy.org")
)

with SessionLocal as sess, sess.begin():
    sandy_addr = sess.scalars(sel_join_stmt).one()

if sandy_addr:
    display(f"Sandy address: {sandy_addr}")

2023-05-27 16:06:32,356 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-27 16:06:32,359 INFO sqlalchemy.engine.Engine SELECT address.id, address.user_id, address.email_address 
FROM address JOIN user ON user.id = address.user_id 
WHERE user.name = ? AND address.email_address = ?
2023-05-27 16:06:32,360 INFO sqlalchemy.engine.Engine [generated in 0.00130s] ('sandy', 'sandy@sqlalchemy.org')
2023-05-27 16:06:32,364 INFO sqlalchemy.engine.Engine COMMIT


'Sandy address: <__main__.Address object at 0x7f611631ef20>'

In [689]:
## Make changes to object
#  https://docs.sqlalchemy.org/en/20/orm/quickstart.html#make-changes
update_stmt = sa.select(User).where(User.name == "patrick")

with SessionLocal as sess:
    patrick = sess.scalars(update_stmt).one()
    patrick.addresses.append(Address(email_address="patrickstar@sqlalchemy.org"))

    sandy_addr = "sandy_cheeks@sqlalchemy.org"

    sess.commit()

2023-05-27 16:06:32,401 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-27 16:06:32,404 INFO sqlalchemy.engine.Engine SELECT user.id, user.name, user.fullname, user.nickname, user.created_at 
FROM user 
WHERE user.name = ?
2023-05-27 16:06:32,407 INFO sqlalchemy.engine.Engine [generated in 0.00292s] ('patrick',)
2023-05-27 16:06:32,410 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2023-05-27 16:06:32,411 INFO sqlalchemy.engine.Engine [cached since 0.1185s ago] (3,)
2023-05-27 16:06:32,416 INFO sqlalchemy.engine.Engine INSERT INTO address (user_id, email_address) VALUES (?, ?)
2023-05-27 16:06:32,417 INFO sqlalchemy.engine.Engine [generated in 0.00110s] (3, 'patrickstar@sqlalchemy.org')
2023-05-27 16:06:32,421 INFO sqlalchemy.engine.Engine COMMIT


In [690]:
## Delete
#  https://docs.sqlalchemy.org/en/20/orm/quickstart.html#some-deletes

with SessionLocal as sess, sess.begin():
    sandy = sess.get(User, 2)

    ## Flush session, delete address before closing session
    sess.flush()

    ## Remove user Patrick
    sess.delete(patrick)

    ## Commit changes
    sess.commit()

2023-05-27 16:06:32,463 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-27 16:06:32,467 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname, user.created_at AS user_created_at 
FROM user 
WHERE user.id = ?
2023-05-27 16:06:32,469 INFO sqlalchemy.engine.Engine [generated in 0.00172s] (2,)
2023-05-27 16:06:32,473 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname, user.created_at AS user_created_at 
FROM user 
WHERE user.id = ?
2023-05-27 16:06:32,474 INFO sqlalchemy.engine.Engine [generated in 0.00110s] (3,)
2023-05-27 16:06:32,476 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.user_id AS address_user_id, address.email_address AS address_email_address 
FROM address 
WHERE ? = address.user_id
2023-05-27 16:06:32,477 INFO sqlalchemy.engine.Engine [cached since 0.1838s ago] (3,